In [1]:
from collections import Counter
from pathlib import Path

import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
data = pd.read_csv("data/mushrooms.csv")
data

In [ ]:
labels_data = list(data["class"])
print(labels_data)

In [ ]:
labels = []
for label in labels_data:
    if label == "p":
        labels.append([0])
    elif label == "e":
        labels.append([1])
        
labels = np.array(labels)
labels.shape

In [ ]:
data = data.drop(labels=["class"], axis=1)
data

In [ ]:
data_test_matrix = []

for column in data.columns:
    column_data = []
    column_values = list(data[column])
    
    counter = Counter(column_values)
    n_column_labels = len(counter)
    column_labels = list(counter.keys())
    
    for i in range(n_column_labels):
        column_data.append(np.zeros(len(column_values)))
        
    for j in range(len(column_values)):
        label_index = column_labels.index(column_values[j])
        column_data[label_index][j] = 1
        
    for data_column in column_data:
        data_test_matrix.append(data_column)
        
data_test_matrix = np.array(data_test_matrix).transpose()
data_test_matrix

In [ ]:
data_test_matrix.shape

In [ ]:
s = np.arange(data_test_matrix.shape[0])
np.random.shuffle(s)
s

In [ ]:
data_test_matrix_shuffled = data_test_matrix[s]
data_test_matrix_shuffled

In [ ]:
labels_shuffled = labels[s]
labels_shuffled

In [ ]:
data_split = 0.1
split_instances = round(data_test_matrix_shuffled.shape[0] * data_split)
training_instances = data_test_matrix_shuffled.shape[0] - split_instances * 2
print("Training instances: {}".format(training_instances))
print("Split instances: {}".format(split_instances))

In [ ]:
training_data = data_test_matrix_shuffled[:training_instances, :]
training_labels = labels_shuffled[:training_instances, :]
training_data.shape

In [ ]:
validation_data = data_test_matrix_shuffled[training_instances:training_instances + split_instances, :]
validation_labels = labels_shuffled[training_instances:training_instances + split_instances, :]
validation_data.shape

In [ ]:
test_data = data_test_matrix_shuffled[training_instances + split_instances:training_instances + split_instances * 2, :]
test_labels = labels_shuffled[training_instances + split_instances:training_instances + split_instances *2, :]
test_data.shape

In [ ]:
Path("training_data").mkdir(exist_ok=True)
np.save(open("training_data/mushrooms_training_data.npy", 'wb'), training_data)
np.save(open("training_data/mushrooms_training_labels.npy", 'wb'), training_labels)
np.save(open("training_data/mushrooms_validation_data.npy", 'wb'), validation_data)
np.save(open("training_data/mushrooms_validation_labels.npy", 'wb'), validation_labels)
np.save(open("training_data/mushrooms_test_data.npy", 'wb'), test_data)
np.save(open("training_data/mushrooms_test_labels.npy", 'wb'), test_labels)